In [12]:
#!pip3.10 install krippendorff
#!pip3.10 install statsmodels
#!pip3.10 install crowd-kit

In [7]:
import json
import copy
import pandas as pd
import re
from statsmodels.stats.inter_rater import fleiss_kappa, aggregate_raters
import krippendorff as kd
import numpy as np

In [8]:
# from google.colab import drive

# drive.mount('/content/gdrive')

In [13]:
df = pd.read_csv("data/assignments_from_pool_42774131__05-02-2024.tsv", sep="\t")
df = df[df['ASSIGNMENT:status']=='APPROVED']

In [14]:
from crowdkit.metrics.data._classification import alpha_krippendorff

In [16]:
tasks = {}
tasks_labels = []
for index, row in df.iterrows():
    tasks_labels.append({'task': row['ASSIGNMENT:task_id'], 'worker': row['ASSIGNMENT:worker_id'], 'label': int(row['OUTPUT:R-4'])})
    if row["ASSIGNMENT:task_id"] not in tasks:
        tasks[row["ASSIGNMENT:task_id"]] = {}
        tasks[row["ASSIGNMENT:task_id"]]["text"] = row["INPUT:input"]
        tasks[row["ASSIGNMENT:task_id"]]["workers"] = {row["ASSIGNMENT:worker_id"]:row["OUTPUT:R-4"]}
    else:
        tasks[row["ASSIGNMENT:task_id"]]["workers"][row["ASSIGNMENT:worker_id"]] = row["OUTPUT:R-4"]

In [27]:
from collections import Counter

toloker2count = dict(Counter(pd.DataFrame.from_dict(tasks_labels).worker.values.tolist()).most_common())

In [28]:
import json
with open("data/output_annotation.json", 'w') as w:
    json.dump(tasks, w)

In [18]:
pd.DataFrame.from_records(tasks_labels)

,task,worker,label
0,00028cae73--65bfa3f40cadac2361249abc,d7c8ba18b024aadad3c03dc12bceface,2
1,00028cae73--65bfa3a1a15018044b04914f,d7c8ba18b024aadad3c03dc12bceface,2
2,00028cae73--65bfa3f80cadac236124a05f,d7c8ba18b024aadad3c03dc12bceface,2
3,00028cae73--65bfa3f30cadac23612498c8,d7c8ba18b024aadad3c03dc12bceface,2
4,00028cae73--65bfa3f50cadac2361249b93,d7c8ba18b024aadad3c03dc12bceface,2
...,...,...,...
6404,00028cae73--65bfa3f80cadac236124a041,8dc6cecab5a1e798269b109e83f64621,2
6405,00028cae73--65bfa3f30cadac236124994e,8dc6cecab5a1e798269b109e83f64621,2
6406,00028cae73--65bfa3f70cadac2361249f6e,8dc6cecab5a1e798269b109e83f64621,2
6407,00028cae73--65bfa3f70cadac2361249e1f,8dc6cecab5a1e798269b109e83f64621,2


In [19]:
alpha_krippendorff(pd.DataFrame.from_records(tasks_labels))

0.2686509960442679

In [29]:
from collections import Counter

js = []
for i,j in Counter(pd.DataFrame.from_dict(tasks_labels).task.values.tolist()).most_common():
    js.append(j)

Counter(js)

Counter({3: 1709,
         66: 4,
         72: 2,
         63: 2,
         62: 2,
         59: 2,
         76: 1,
         71: 1,
         70: 1,
         68: 1,
         61: 1,
         60: 1,
         51: 1,
         49: 1})

In [21]:
from nltk.tokenize import NLTKWordTokenizer

In [30]:
tags = {1.0: 1, 2.0: 2, 3.0: 3, 4.0: 4, 0.0: 0}

In [31]:
import pandas as pd

# Placeholder for the new rows
rows = []

for n_sent, task in enumerate(tasks):
    text = tasks[task]['text']
    workers = tasks[task]['workers']
    row = {'text': text, 'n_sent': n_sent}
    
    for worker, label in workers.items():
        row[worker] = label
    
    rows.append(row)

# Create the DataFrame at once
df_raters = pd.DataFrame(rows)

# (Optional) Ensure the 'n_sent' column is the correct type
df_raters = df_raters.astype({'n_sent': 'int32'})

In [32]:
df_raters

,text,n_sent,d7c8ba18b024aadad3c03dc12bceface,31e4bb0217a21ef6451faaee8847d7c1,b0e385e880cb3f5033308f1a9d63e5c4,9a8928d0d44faa52cf9e7118122d68a3,646ef569491540d55e1f3140841a1a15,2273fc122c3fd60f9e096cf0afaeabb9,5f1770836609d188639b9b73f25500d8,a7cda650641a4271fba8f12bceef3cbf,...,9f92499d14c6fdba6881e7dc9e6edb97,6c0aed0f589955fe8a49de77e935f62c,d04cf36ea1950c41c1477f00bd29e0a2,0cf8c7a691d536b732042ff3db2f1023,2e705095056dec78df05515f62656880,4e68b29b49a21c8f55390154db8e2756,7a91a16c8662352508c0f2cfde2f6f42,b3d4859e6d92a76c007fc8e7d9b2aa92,355e2bf3c1d973f919f3a979d966b25c,7d77e5454ea232d0be01868a0b4be267
0,**Family Guy** vs **The Simpsons**\r\n\r\n*Rat...,0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,**Cement** vs **Concrete**\r\n\r\n*Rate the se...,1,2.0,NaN,NaN,3.0,2.0,3.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,**chocolate** vs **tea**\r\n\r\n*Rate the sent...,2,2.0,3.0,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,**BMW** vs **Audi**\r\n\r\n*Rate the sentence ...,3,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,**kids** vs **adults**\r\n\r\n*Rate the senten...,4,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1724,**concrete** vs **wood**\r\n\r\n*Rate the sent...,1724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1725,**perl** vs **python**\r\n\r\n*Rate the senten...,1725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1726,**skiing** vs **snowboarding**\r\n\r\n*Rate th...,1726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1727,**baseball** vs **softball**\r\n\r\n*Rate the ...,1727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Count Fleiss' kappa

In [33]:
sentence = df_raters.loc[:, df_raters.columns != 'text']

### Here we select the annotators

In [34]:
annotators = list(df_raters.columns)[2:]
sentence = sentence[annotators]

In [36]:
sentence = sentence[annotators]
all_data = sentence[~sentence.isnull().any(axis=1)]
# print(all_data)
if len(all_data) > 0:
    print(fleiss_kappa(aggregate_raters(all_data)[0], method='fleiss'))

## Count Krippendorf's Alpha

In [37]:
sentence.T

,0,1,2,3,4,5,6,7,8,9,...,1719,1720,1721,1722,1723,1724,1725,1726,1727,1728
d7c8ba18b024aadad3c03dc12bceface,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31e4bb0217a21ef6451faaee8847d7c1,2.0,NaN,3.0,3.0,2.0,NaN,NaN,3.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b0e385e880cb3f5033308f1a9d63e5c4,2.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9a8928d0d44faa52cf9e7118122d68a3,NaN,3.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
646ef569491540d55e1f3140841a1a15,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4e68b29b49a21c8f55390154db8e2756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7a91a16c8662352508c0f2cfde2f6f42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b3d4859e6d92a76c007fc8e7d9b2aa92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
355e2bf3c1d973f919f3a979d966b25c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
kd.alpha(sentence.T, level_of_measurement='interval')

0.3350446915451717

In [39]:
sentence

,d7c8ba18b024aadad3c03dc12bceface,31e4bb0217a21ef6451faaee8847d7c1,b0e385e880cb3f5033308f1a9d63e5c4,9a8928d0d44faa52cf9e7118122d68a3,646ef569491540d55e1f3140841a1a15,2273fc122c3fd60f9e096cf0afaeabb9,5f1770836609d188639b9b73f25500d8,a7cda650641a4271fba8f12bceef3cbf,ad486b05c2371539edf2f43c139db430,437cb936254a18312f9c5cb2ef17310b,...,9f92499d14c6fdba6881e7dc9e6edb97,6c0aed0f589955fe8a49de77e935f62c,d04cf36ea1950c41c1477f00bd29e0a2,0cf8c7a691d536b732042ff3db2f1023,2e705095056dec78df05515f62656880,4e68b29b49a21c8f55390154db8e2756,7a91a16c8662352508c0f2cfde2f6f42,b3d4859e6d92a76c007fc8e7d9b2aa92,355e2bf3c1d973f919f3a979d966b25c,7d77e5454ea232d0be01868a0b4be267
0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,NaN,NaN,3.0,2.0,3.0,3.0,3.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Count Cohen's kappa

In [40]:
from sklearn.metrics import cohen_kappa_score
import itertools

In [ ]:
cohen = []

for a1, a2 in itertools.combinations(annotators, 2):
    sentence = df_raters[[a1, a2]]
    all_data = sentence[~sentence.isnull().any(axis=1)]
    list1 = [tags[i] for i in all_data[a1].values.tolist()]
    list2 = [tags[i] for i in all_data[a2].values.tolist()]
    if str(cohen_kappa_score(list1, list2, labels=[0,1,2,3,4])) != 'nan':
        cohen.append(cohen_kappa_score(list1, list2, labels=[0,1,2,3,4]))

In [42]:
np.mean(cohen), np.median(cohen)

(0.16959827587164003, 0.0)

In [43]:
from collections import Counter

In [44]:
c = 0
with open("labelling_dataset_ru.tsv", 'w') as w:
    w.write(f"text\ttag\n")
    for _, subset in df_raters.groupby('n_sent'):
        c += 1
        lines = pd.DataFrame(subset).values.tolist()
        for line in lines:
            text = line[0] #.replace("***", "")
            labels = [i for i in line[2:] if not i is np.nan]
            label = Counter(labels).most_common()[0][0]
            w.write(f"{text}\t{label}\n")
c

1729